# **Método símplex:**
## **Integración con Kale y Minikube**

## 1.  Introducción

En la parte 2 de la práctica 1 desarrollamos el paquete ***`mex`*** para resolver problemas de optimización por medio del método símplex.  La documentación del paquete se encuentra [aquí](https://optimizacion-2-2021-1-gh-classroom.github.io/practica-1-segunda-parte-caroacostatovany/).

En esta parte 1 de la práctica 2, integraremos nuestro paquete con Kale y Minikube para poder lanzar pipelines desde un Notebook de Jupyter, y utilizar nuestro paquete para la resolución de optimización.

## 2.  Implementación

Se puede hacer uso del siguiente botón de binder para hacer pruebas con nuestro paquete:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/optimizacion-2-2021-1-gh-classroom/practica-2-primera-parte-caroacostatovany.git/main?urlpath=lab)

Adicionalmente, se creó la imagen de Docker **`optimizacion-2-practica-2-parte-1`** que contiene todo lo necesario para ejecutar nuestro paquete ***`mex`*** junto con Kale.

## 3.  Evaluación de resultados

In [50]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## 3.1. Problema prototipo - Maximización:
(De las notas del Libro de Optimización)

$$\max_{x \in \mathbb{R}^2} 3x_1 + 5x_2$$

$$\text{sujeto a: }$$
$$x_1 \leq 4$$
$$2x_2 \leq 12$$
$$3x_1 + 2x_2 \leq 18$$
$$x_1 \geq 0, x_2 \geq 0$$

### Resolvemos con `scipy`

In [51]:
c_max_obj_1 = [-3, -5]
A_max_obj_1 = [[1, 0], [0, 2], [3, 2]]
b_max_obj_1 = [4, 12, 18]

In [52]:
max_obj_1 = -1*linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).fun

In [53]:
coeff_obj_1 = linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).x

### Resolvemos con `mex`

In [54]:
n_var_approx_1 = 2
n_cons_approx_1 = 3

In [55]:
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)

In [56]:
constrain(matrix_max_approx_1,'1,0,L,4')
constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')
obj(matrix_max_approx_1,'3,5,0')

In [57]:
problem_approx_1 = maxz(matrix_max_approx_1)

In [58]:
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))

In [59]:
print("El valor objetivo obtenido con scipy es: ", max_obj_1)
print("El valor aproximado obtenido con mex es: ", max_approx_1)

El valor objetivo obtenido con scipy es:  35.99999997843391
El valor aproximado obtenido con mex es:  36.0


In [60]:
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_1)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1)

Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


In [61]:
assert max_obj_1 == approx(max_approx_1), "El valor aproximado es incorrecto"
assert coeff_obj_1 == approx(coeff_approx_1), "El valor de los coeficientes aproximados es incorrecto"

## 3.2. Ejemplo del lema de Farkas - Minimización:
(De las notas del Libro de Optimización)

$$\min_{x \in \mathbb{R}^2} -1x_1 + -3x_2$$

$$\text{sujeto a: }$$
$$x_1 + x_2 \leq 6$$
$$-x_1 + 2x_2 \leq 8$$
$$x_1 \geq 0, x_2 \geq 0$$

### Resolvemos con `scipy`

In [62]:
c_min_obj_2 = [-1, -3]
A_min_obj_2 = [[1, 1], [-1, 2]]
b_min_obj_2 = [6, 8]

In [63]:
min_obj_2 = linprog(c_min_obj_2, A_ub = A_min_obj_2, b_ub = b_min_obj_2).fun

In [64]:
coeff_obj_2 = linprog(c_min_obj_2, A_ub = A_min_obj_2, b_ub = b_min_obj_2).x

### Resolvemos con `mex`

In [65]:
n_var_approx_2 = 2
n_cons_approx_2 = 2

In [66]:
matrix_min_approx_2 = create_matrix(n_var_approx_2,n_cons_approx_2)

In [67]:
constrain(matrix_min_approx_2,'1,1,L,6')
constrain(matrix_min_approx_2,'-1,2,L,8')
obj(matrix_min_approx_2,'-1,-3,0')

In [68]:
problem_approx_2 = minz(matrix_min_approx_2)

In [69]:
min_approx_2 = problem_approx_2['min']
problem_approx_2.pop('min')
coeff_approx_2 = np.array(list(problem_approx_2.values()))

In [70]:
print("El valor objetivo obtenido con scipy es: ", min_obj_2)
print("El valor aproximado obtenido con mex es: ", min_approx_2)

El valor objetivo obtenido con scipy es:  -15.333333333015998
El valor aproximado obtenido con mex es:  -15.333333333333332


In [71]:
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_2)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_2)

Los coeficientes objetivos obtenidos con scipy son:  [1.33333333 4.66666667]
Los coeficientes aproximados obtenidos con mex son:  [1.33333333 4.66666667]


In [72]:
assert min_approx_2 == approx(min_obj_2), "El valor aproximado es incorrecto"
assert coeff_obj_2 == approx(coeff_approx_2), "El valor de los coeficientes aproximados es incorrecto"

## 3.3. Ejemplo de problema de flujo con costo mínimo:
(De las notas del Libro de Optimización)

$$ \min 2 x_{AB} + 4 x_{AC} + 9 x_{AD} + 3 x_{BC} + x_{CE} + 3 x_{DE} + 2x_{ED}$$

$$\text{sujeto a: }$$
$$
\begin{eqnarray}
&x_{AB}&  + &x_{AC}& + &x_{AD}&   &&         &&         &&         &&       &=& 50 \nonumber \\
&-x_{AB}&   &&         &&       + &x_{BC}&   &&         &&         &&       &=& 40 \nonumber \\
&&        - &x_{AC}&   &&       - &x_{BC}& + &x_{CE}&   &&         &&       &=& 0 \nonumber \\
&&          &&       - &x_{AD}&   &&         &&       + &x_{DE}& - &x_{ED}& &=& -30 \nonumber \\
&&          &&         &&         &&       - &x_{CE}& - &x_{DE}& + &x_{ED}& &=& -60 \nonumber
\end{eqnarray}
$$

### Resolvemos con `scipy`

In [73]:
c_net_obj_3 = [2,4,9,3,1,3,2]
A_ub_net_obj_3 = [[1,0,0,0,0,0,0],
            [0,0,0,0,1,0,0]]
b_ub_net_obj_3 = [10,80]
A_eq_net_obj_3 = [[1,1,1,0,0,0,0],
         [-1,0,0,1,0,0,0],
         [0,-1,0,-1,1,0,0],
         [0,0,-1,0,0,1,-1],
         [0,0,0,0,-1,-1,1]]
b_eq_net_obj_3 = [50,40,0,-30,-60]

In [74]:
net_obj_3 = linprog(c_net_obj_3, A_ub=A_ub_net_obj_3, b_ub=b_ub_net_obj_3, A_eq=A_eq_net_obj_3, b_eq=b_eq_net_obj_3).fun

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


In [75]:
net_coeff_obj_3 = linprog(c_net_obj_3, A_ub=A_ub_net_obj_3, b_ub=b_ub_net_obj_3, A_eq=A_eq_net_obj_3, b_eq=b_eq_net_obj_3).x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


### Resolvemos con `mex`

In [76]:
n_var_approx_3 = 7
n_cons_approx_3 = 7

In [77]:
matrix_net_approx_3 = create_matrix(n_var_approx_3,n_cons_approx_3)

In [78]:
constrain(matrix_net_approx_3,'1,1,1,0,0,0,0,E,50')
constrain(matrix_net_approx_3,'-1,0,0,1,0,0,0,E,40')
constrain(matrix_net_approx_3,'0,-1,0,-1,1,0,0,E,0')
constrain(matrix_net_approx_3,'0,0,-1,0,0,1,-1,E,-30')
constrain(matrix_net_approx_3,'0,0,0,0,-1,-1,1,E,-60')
constrain(matrix_net_approx_3,'1,0,0,0,0,0,0,L,10')
constrain(matrix_net_approx_3,'0,0,0,0,1,0,0,L,80')
obj(matrix_net_approx_3,'2,4,9,3,1,3,2,0')

In [79]:
problem_approx_3 = minz(matrix_net_approx_3)

In [80]:
net_approx_3 = problem_approx_3['min']
problem_approx_3.pop('min')
net_coeff_approx_3 = np.array(list(problem_approx_3.values()))

In [81]:
print("El valor objetivo obtenido con scipy es: ", net_obj_3)
print("El valor aproximado obtenido con mex es: ", net_approx_3)

El valor objetivo obtenido con scipy es:  489.99999837231917
El valor aproximado obtenido con mex es:  490.0


In [82]:
print("Los coeficientes objetivos obtenidos con scipy son: ", np.round(net_coeff_obj_3))
print("Los coeficientes aproximados obtenidos con mex son: ", net_coeff_approx_3)

Los coeficientes objetivos obtenidos con scipy son:  [ 0. 40. 10. 40. 80.  0. 20.]
Los coeficientes aproximados obtenidos con mex son:  [ 0. 40. 10. 40. 80.  0. 20.]


In [83]:
assert net_obj_3 == approx(net_approx_3), "El valor aproximado es incorrecto"
assert np.round(net_coeff_obj_3,0) == approx(net_coeff_approx_3), "El valor de los coeficientes aproximados es incorrecto"